<a href="https://colab.research.google.com/github/yoonjaeuk/2023S-Ajou-ML/blob/main/alopecia_ResNet.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#!pip install thop

In [ ]:
#from thop import profile
#from thop import clever_format
#from torchsummary import summary

In [ ]:
import torchvision
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision.transforms as transforms
import os
import torchvision.models as models

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import torchvision.datasets as datasets
import torchvision.transforms  as transforms

data_folder = '/content/drive/MyDrive/팀플/탈모데이터/Training'
transform = transforms.Compose([
    transforms.Resize((100,100)),
    transforms.ToTensor()
])



class_to_idx = {
    '0.양호': 0,
    '1.경증': 1,
    '2.중등도':2,
    '3.중증':3
}
trainset = datasets.DatasetFolder(
    root = data_folder,
    loader =datasets.folder.default_loader,
    extensions = ('jpg', 'jpeg', 'png'),
    transform =transform
)


In [ ]:
len(trainset)

8179

In [ ]:
data_folder = '/content/drive/MyDrive/팀플/탈모데이터/Validation'
transform = transforms.Compose([
    transforms.Resize((100,100)),
    transforms.ToTensor()
])

class_to_idx = {
    '0.양호': 0,
    '1.경증': 1,
    '2.중등도':2,
    '3.중증':3
}
validationset = datasets.DatasetFolder(
    root = data_folder,
    loader =datasets.folder.default_loader,
    extensions = ('jpg', 'jpeg', 'png'),
    transform =transform
)

In [ ]:
len(validationset)

3347

In [ ]:
import torch.utils.data as data
train_loader = torch.utils.data.DataLoader(trainset, batch_size=32, shuffle=True, num_workers=2) # trainset이 미리 준비되어 있어야함
#Batch_size는 하이퍼 파라미터

In [ ]:
val_loader = torch.utils.data.DataLoader(trainset, batch_size=32, shuffle=True, num_workers=2) # trainset이 미리 준비되어 있어야함

In [ ]:
# 제대로 폴더명과 사진이 할당되었는지 확인하는 코드
'''class_to_idx = trainset.class_to_idx
samples =  trainset.samples

for img_path, target in samples:
    class_name = list(class_to_idx.keys())[list(class_to_idx.values()).index(target)]
    print(f"Image path: {img_path}, Target: {class_name}")'''

'class_to_idx = trainset.class_to_idx\nsamples =  trainset.samples\n\nfor img_path, target in samples:\n    class_name = list(class_to_idx.keys())[list(class_to_idx.values()).index(target)]\n    print(f"Image path: {img_path}, Target: {class_name}")'

In [ ]:
# 제대로 폴더명과 사진이 할당되었는지 확인하는 코드
'''class_to_idx = validationset.class_to_idx
samples =  validationset.samples

for img_path, target in samples:
    class_name = list(class_to_idx.keys())[list(class_to_idx.values()).index(target)]
    print(f"Image path: {img_path}, Target: {class_name}")'''

'class_to_idx = validationset.class_to_idx\nsamples =  validationset.samples\n\nfor img_path, target in samples:\n    class_name = list(class_to_idx.keys())[list(class_to_idx.values()).index(target)]\n    print(f"Image path: {img_path}, Target: {class_name}")'

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F

In [ ]:
use_cuda = torch.cuda.is_available()
print(use_cuda)

True


In [ ]:
print("Torch version:{}".format(torch.__version__))
print("cuda version: {}".format(torch.version.cuda))
print("cudnn version:{}".format(torch.backends.cudnn.version()))

Torch version:2.0.1+cu118
cuda version: 11.8
cudnn version:8700


In [ ]:
# Simple Learning Rate Scheduler
def lr_scheduler(optimizer, epoch):
    lr = learning_rate # 하이퍼 파라미터
    if epoch >= 50:
        lr /= 10
    if epoch >= 100:
        lr /= 10
    for param_group in optimizer.param_groups:
        param_group['lr'] = lr

# Xavier
def init_weights(m):
    if isinstance(m, nn.Linear):
        torch.nn.init.xavier_uniform(m.weight)
        m.bias.data.fill_(0.01)

In [ ]:
class BasicBlock(nn.Module):
  # mul은 ResNet18, ResNet34, ResNet50, ResNet101, ResNet152등 구조 생성에 사용됨
  mul = 1
  def __init__(self, in_planes, out_planes, stride = 1):
    super(BasicBlock, self).__init__()

    # stride를 통해 width, height 조정
    self.conv1 = nn.Conv2d(in_planes, out_planes, kernel_size =3, stride = stride, padding =1, bias = False)
    self.bn1 = nn.BatchNorm2d(out_planes)

    # stride = 1, padding = 1 --> width, height 항시 유지됨
    self.conv2 = nn.Conv2d(out_planes, out_planes, kernel_size =3, stride = 1, padding = 1, bias = False)
    self.bn2 = nn.BatchNorm2d(out_planes)

    # shrotcut 구현
    self.shortcut = nn.Sequential()

    if stride !=1:
      self.shortcut = nn.Sequential(
          nn.Conv2d(in_planes, out_planes, kernel_size = 1, stride = stride, bias =False),
          nn.BatchNorm2d(out_planes)
          )

  def forward(self,x):
    out = self.conv1(x)
    out = self.bn1(out)
    out = F.relu(out)
    out = self.conv2(out)
    out = self.bn2(out)
    out += self.shortcut(x)
    out = F.relu(out)
    return out

In [ ]:
# Basic Residual Block에서 컨볼루션 방식이 달라졌을 뿐 크게 다를것은 없음
class BottleNeck(nn.Module):
  # mul은 4  즉 64 -> 256
  mul=4
  def __init__(self, in_planes, out_planes, stride = 1):
    super(BottleNeck, self).__init__()

    #first convolution은 widht와 height를 downsampling 즉 width와 height를 줄임
    self.conv1 = nn.Conv2d(in_planes, out_planes, kernel_size = 1, stride = stride, bias = False)
    self.bn1 = nn.BatchNorm2d(out_planes)

    self.conv2 = nn.Conv2d(out_planes, out_planes, kernel_size = 3, stride = 1, padding = 1, bias = False)
    self.bn2 = nn.BatchNorm2d(out_planes)

    self.conv3 = nn.Conv2d(out_planes, out_planes*self.mul, kernel_size = 1, stride = 1, bias = False)
    self.bn3 = nn.BatchNorm2d(out_planes*self.mul)

    self.shortcut = nn.Sequential()

    if stride != 1 or in_planes != out_planes*self.mul:
      # stride 가 1이 아니거나 또는 in_plnanes 과 out_planes*self.mul 이 다르다면
      self.shortcut = nn.Sequential(
          nn.Conv2d(in_planes, out_planes*self.mul, kernel_size = 1 , stride = stride, bias =False),
          nn.BatchNorm2d(out_planes*self.mul)
      )

  def forward(self,x):
    out = self.conv1(x)
    out = self.bn1(out)
    out = F.relu(out)
    out = self.conv2(out)
    out = self.bn2(out)
    out = F.relu(out)
    out = self.conv3(out)
    out = self.bn3(out)
    out += self.shortcut(x)
    out = F.relu(out)
    return out

In [ ]:
class ResNet(nn.Module):
  def __init__(self, block, num_blocks, num_classes = 4):
    super(ResNet, self).__init__()

    self.in_planes =64 # feature의 채널 수 하이퍼 파라미터

    self.conv1 = nn.Conv2d(3,self.in_planes, kernel_size = 7, stride = 2, padding = 3)
    self.bn1 = nn.BatchNorm2d(self.in_planes)
    self.maxpool1 = nn.MaxPool2d(kernel_size =3, stride = 2, padding =1)

    self.layer1 = self.make_layer(block,64,num_blocks[0], stride = 1)
    self.layer2 = self.make_layer(block,128,num_blocks[1], stride = 2)
    self.layer3 = self.make_layer(block,256,num_blocks[2], stride = 2)
    self.layer4 = self.make_layer(block,512,num_blocks[3], stride = 2)
    self.avgpool = nn.AdaptiveAvgPool2d((1,1))

    self.linear =nn.Linear(512*block.mul, num_classes)


  # 다양한 아키텍쳐 생성을 위해 make_layer로 Sequential 생성
  def make_layer(self, block, out_planes, num_blocks, stride):
    # layer 앞부분에서만 크기를 절반으로 줄이므로, 아래와 같은 구조
    strides = [stride] + [1]*(num_blocks-1)
    layers = []
    for i in range(num_blocks):
      layers.append(block(self.in_planes, out_planes, strides[i]))
      self.in_planes = block.mul*out_planes
    return nn.Sequential(*layers)

  def forward(self, x):
    out = self.conv1(x)
    out = self.bn1(out)
    out = F.relu(out)
    out = self.maxpool1(out)
    out = self.layer1(out)
    out = self.layer2(out)
    out = self.layer3(out)
    out = self.layer4(out)
    out = self.avgpool(out)
    out = torch.flatten(out,1)
    out = self.linear(out)
    return out

In [ ]:
def ResNet18():
  return ResNet(BasicBlock, [2,2,2,2])
def ResNet50():
  return ResNet(BottleNeck, [3,4,6,3])

In [ ]:
if torch.cuda.is_available():
    device = torch.device("cuda")
    print("cuda")
else:
    device = torch.device("cpu")

model = ResNet18()

cuda


In [ ]:
model.apply(init_weights)
model = model.to(device)

<ipython-input-16-db4fc92bf600>:14: UserWarning: nn.init.xavier_uniform is now deprecated in favor of nn.init.xavier_uniform_.
  torch.nn.init.xavier_uniform(m.weight)


In [ ]:
learning_rate = 0.01 # 하이퍼 파라미터
num_epoch = 20 # 하이퍼 파라미터
model_name = 'model.pth'

loss_fn = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=learning_rate, weight_decay=0.0001)
# optimizer 하이퍼 파라미터
train_loss = 0
valid_loss = 0
correct = 0
total_cnt = 0
best_acc = 0
acc_record = []

In [ ]:
# Train
for epoch in range(num_epoch):
    print(f"====== { epoch+1} epoch of { num_epoch } ======")
    model.train()
    lr_scheduler(optimizer, epoch)
    train_loss = 0
    valid_loss = 0
    correct = 0
    total_cnt = 0


    for step, batch in enumerate(train_loader):
        #  input and target
        batch[0], batch[1] = batch[0].to(device), batch[1].to(device)
        optimizer.zero_grad()


        logits = model(batch[0])
        loss = loss_fn(logits, batch[1])
        loss.backward()

        optimizer.step()
        train_loss += loss.item()
        _, predict = logits.max(1)

        total_cnt += batch[1].size(0)
        correct +=  predict.eq(batch[1]).sum().item()

        if step%len(train_loader) ==0:
            #print(f"\n====== { step } Step of { len(train_loader) } ======")
            accuracy = correct / total_cnt
            print(f"Train Acc : {accuracy }")
            acc_record.append(accuracy)
            print(f"Train Loss : { loss.item() / batch[1].size(0) }")

        print(f"Batch {step+1} of {len(train_loader)}")

        if step == len(train_loader) -1 and len(batch[0] < 32):
          pass

    correct = 0
    total_cnt = 0

스트리밍 출력 내용이 길어서 마지막 5000줄이 삭제되었습니다.
Batch 178 of 256
Batch 179 of 256
Batch 180 of 256
Batch 181 of 256
Batch 182 of 256
Batch 183 of 256
Batch 184 of 256
Batch 185 of 256
Batch 186 of 256
Batch 187 of 256
Batch 188 of 256
Batch 189 of 256
Batch 190 of 256
Batch 191 of 256
Batch 192 of 256
Batch 193 of 256
Batch 194 of 256
Batch 195 of 256
Batch 196 of 256
Batch 197 of 256
Batch 198 of 256
Batch 199 of 256
Batch 200 of 256
Batch 201 of 256
Batch 202 of 256
Batch 203 of 256
Batch 204 of 256
Batch 205 of 256
Batch 206 of 256
Batch 207 of 256
Batch 208 of 256
Batch 209 of 256
Batch 210 of 256
Batch 211 of 256
Batch 212 of 256
Batch 213 of 256
Batch 214 of 256
Batch 215 of 256
Batch 216 of 256
Batch 217 of 256
Batch 218 of 256
Batch 219 of 256
Batch 220 of 256
Batch 221 of 256
Batch 222 of 256
Batch 223 of 256
Batch 224 of 256
Batch 225 of 256
Batch 226 of 256
Batch 227 of 256
Batch 228 of 256
Batch 229 of 256
Batch 230 of 256
Batch 231 of 256
Batch 232 of 256
Batch 233 of 256
Batch 234 of

In [ ]:
# Train
learning_rate = 0.005
for epoch in range(20):
    print(f"====== { epoch+1} epoch of { 20 } ======")
    model.train()
    lr_scheduler(optimizer, epoch)
    train_loss = 0
    valid_loss = 0
    correct = 0
    total_cnt = 0


    for step, batch in enumerate(train_loader):
        #  input and target
        batch[0], batch[1] = batch[0].to(device), batch[1].to(device)
        optimizer.zero_grad()


        logits = model(batch[0])
        loss = loss_fn(logits, batch[1])
        loss.backward()

        optimizer.step()
        train_loss += loss.item()
        _, predict = logits.max(1)

        total_cnt += batch[1].size(0)
        correct +=  predict.eq(batch[1]).sum().item()

        if step%len(train_loader) ==0:
            #print(f"\n====== { step } Step of { len(train_loader) } ======")
            accuracy = correct / total_cnt
            print(f"Train Acc : {accuracy }")
            acc_record.append(accuracy)
            print(f"Train Loss : { loss.item() / batch[1].size(0) }")

        print(f"Batch {step+1} of {len(train_loader)}")

        if step == len(train_loader) -1 and len(batch[0] < 32):
          pass

    correct = 0
    total_cnt = 0

스트리밍 출력 내용이 길어서 마지막 5000줄이 삭제되었습니다.
Batch 178 of 256
Batch 179 of 256
Batch 180 of 256
Batch 181 of 256
Batch 182 of 256
Batch 183 of 256
Batch 184 of 256
Batch 185 of 256
Batch 186 of 256
Batch 187 of 256
Batch 188 of 256
Batch 189 of 256
Batch 190 of 256
Batch 191 of 256
Batch 192 of 256
Batch 193 of 256
Batch 194 of 256
Batch 195 of 256
Batch 196 of 256
Batch 197 of 256
Batch 198 of 256
Batch 199 of 256
Batch 200 of 256
Batch 201 of 256
Batch 202 of 256
Batch 203 of 256
Batch 204 of 256
Batch 205 of 256
Batch 206 of 256
Batch 207 of 256
Batch 208 of 256
Batch 209 of 256
Batch 210 of 256
Batch 211 of 256
Batch 212 of 256
Batch 213 of 256
Batch 214 of 256
Batch 215 of 256
Batch 216 of 256
Batch 217 of 256
Batch 218 of 256
Batch 219 of 256
Batch 220 of 256
Batch 221 of 256
Batch 222 of 256
Batch 223 of 256
Batch 224 of 256
Batch 225 of 256
Batch 226 of 256
Batch 227 of 256
Batch 228 of 256
Batch 229 of 256
Batch 230 of 256
Batch 231 of 256
Batch 232 of 256
Batch 233 of 256
Batch 234 of

In [ ]:
# Train
learning_rate = 0.005
for epoch in range(10):
    print(f"====== { epoch+1} epoch of { 10 } ======")
    model.train()
    lr_scheduler(optimizer, epoch)
    train_loss = 0
    valid_loss = 0
    correct = 0
    total_cnt = 0


    for step, batch in enumerate(train_loader):
        #  input and target
        batch[0], batch[1] = batch[0].to(device), batch[1].to(device)
        optimizer.zero_grad()


        logits = model(batch[0])
        loss = loss_fn(logits, batch[1])
        loss.backward()

        optimizer.step()
        train_loss += loss.item()
        _, predict = logits.max(1)

        total_cnt += batch[1].size(0)
        correct +=  predict.eq(batch[1]).sum().item()

        if step%len(train_loader) ==0:
            #print(f"\n====== { step } Step of { len(train_loader) } ======")
            accuracy = correct / total_cnt
            print(f"Train Acc : {accuracy }")
            acc_record.append(accuracy)
            print(f"Train Loss : { loss.item() / batch[1].size(0) }")

        print(f"Batch {step+1} of {len(train_loader)}")

        if step == len(train_loader) -1 and len(batch[0] < 32):
          pass

    correct = 0
    total_cnt = 0

In [ ]:
# Train
learning_rate = 0.0025
for epoch in range(10):
    print(f"====== { epoch+1} epoch of { 10 } ======")
    model.train()
    lr_scheduler(optimizer, epoch)
    train_loss = 0
    valid_loss = 0
    correct = 0
    total_cnt = 0


    for step, batch in enumerate(train_loader):
        #  input and target
        batch[0], batch[1] = batch[0].to(device), batch[1].to(device)
        optimizer.zero_grad()


        logits = model(batch[0])
        loss = loss_fn(logits, batch[1])
        loss.backward()

        optimizer.step()
        train_loss += loss.item()
        _, predict = logits.max(1)

        total_cnt += batch[1].size(0)
        correct +=  predict.eq(batch[1]).sum().item()

        if step%len(train_loader) ==0:
            #print(f"\n====== { step } Step of { len(train_loader) } ======")
            accuracy = correct / total_cnt
            print(f"Train Acc : {accuracy }")
            acc_record.append(accuracy)
            print(f"Train Loss : { loss.item() / batch[1].size(0) }")

        print(f"Batch {step+1} of {len(train_loader)}")

        if step == len(train_loader) -1 and len(batch[0] < 32):
          pass

    correct = 0
    total_cnt = 0

====== 1 epoch of 10 ======
Train Acc : 0.78125
Train Loss : 0.01561235636472702
Batch 1 of 256
Batch 2 of 256
Batch 3 of 256
Batch 4 of 256
Batch 5 of 256
Batch 6 of 256
Batch 7 of 256
Batch 8 of 256
Batch 9 of 256
Batch 10 of 256
Batch 11 of 256
Batch 12 of 256
Batch 13 of 256
Batch 14 of 256
Batch 15 of 256
Batch 16 of 256
Batch 17 of 256
Batch 18 of 256
Batch 19 of 256
Batch 20 of 256
Batch 21 of 256
Batch 22 of 256
Batch 23 of 256
Batch 24 of 256
Batch 25 of 256
Batch 26 of 256
Batch 27 of 256
Batch 28 of 256
Batch 29 of 256
Batch 30 of 256
Batch 31 of 256
Batch 32 of 256
Batch 33 of 256
Batch 34 of 256
Batch 35 of 256
Batch 36 of 256
Batch 37 of 256
Batch 38 of 256
Batch 39 of 256
Batch 40 of 256
Batch 41 of 256
Batch 42 of 256
Batch 43 of 256
Batch 44 of 256
Batch 45 of 256
Batch 46 of 256
Batch 47 of 256
Batch 48 of 256
Batch 49 of 256
Batch 50 of 256
Batch 51 of 256
Batch 52 of 256
Batch 53 of 256
Batch 54 of 256
Batch 55 of 256
Batch 56 of 256
Batch 57 of 256
Batch 58 of 256


In [ ]:
# Train
learning_rate = 0.0025
for epoch in range(10):
    print(f"====== { epoch+1} epoch of { 10 } ======")
    model.train()
    lr_scheduler(optimizer, epoch)
    train_loss = 0
    valid_loss = 0
    correct = 0
    total_cnt = 0


    for step, batch in enumerate(train_loader):
        #  input and target
        batch[0], batch[1] = batch[0].to(device), batch[1].to(device)
        optimizer.zero_grad()


        logits = model(batch[0])
        loss = loss_fn(logits, batch[1])
        loss.backward()

        optimizer.step()
        train_loss += loss.item()
        _, predict = logits.max(1)

        total_cnt += batch[1].size(0)
        correct +=  predict.eq(batch[1]).sum().item()

        if step%len(train_loader) ==0:
            #print(f"\n====== { step } Step of { len(train_loader) } ======")
            accuracy = correct / total_cnt
            print(f"Train Acc : {accuracy }")
            acc_record.append(accuracy)
            print(f"Train Loss : { loss.item() / batch[1].size(0) }")

        print(f"Batch {step+1} of {len(train_loader)}")

        if step == len(train_loader) -1 and len(batch[0] < 32):
          pass

    correct = 0
    total_cnt = 0

====== 1 epoch of 10 ======
Train Acc : 0.8125
Train Loss : 0.0125122657045722
Batch 1 of 256
Batch 2 of 256
Batch 3 of 256
Batch 4 of 256
Batch 5 of 256
Batch 6 of 256
Batch 7 of 256
Batch 8 of 256
Batch 9 of 256
Batch 10 of 256
Batch 11 of 256
Batch 12 of 256
Batch 13 of 256
Batch 14 of 256
Batch 15 of 256
Batch 16 of 256
Batch 17 of 256
Batch 18 of 256
Batch 19 of 256
Batch 20 of 256
Batch 21 of 256
Batch 22 of 256
Batch 23 of 256
Batch 24 of 256
Batch 25 of 256
Batch 26 of 256
Batch 27 of 256
Batch 28 of 256
Batch 29 of 256
Batch 30 of 256
Batch 31 of 256
Batch 32 of 256
Batch 33 of 256
Batch 34 of 256
Batch 35 of 256
Batch 36 of 256
Batch 37 of 256
Batch 38 of 256
Batch 39 of 256
Batch 40 of 256
Batch 41 of 256
Batch 42 of 256
Batch 43 of 256
Batch 44 of 256
Batch 45 of 256
Batch 46 of 256
Batch 47 of 256
Batch 48 of 256
Batch 49 of 256
Batch 50 of 256
Batch 51 of 256
Batch 52 of 256
Batch 53 of 256
Batch 54 of 256
Batch 55 of 256
Batch 56 of 256
Batch 57 of 256
Batch 58 of 256
Ba

In [ ]:
# Train
learning_rate = 0.0001
for epoch in range(10):
    print(f"====== { epoch+1} epoch of { 10 } ======")
    model.train()
    lr_scheduler(optimizer, epoch)
    train_loss = 0
    valid_loss = 0
    correct = 0
    total_cnt = 0


    for step, batch in enumerate(train_loader):
        #  input and target
        batch[0], batch[1] = batch[0].to(device), batch[1].to(device)
        optimizer.zero_grad()


        logits = model(batch[0])
        loss = loss_fn(logits, batch[1])
        loss.backward()

        optimizer.step()
        train_loss += loss.item()
        _, predict = logits.max(1)

        total_cnt += batch[1].size(0)
        correct +=  predict.eq(batch[1]).sum().item()

        if step%len(train_loader) ==0:
            #print(f"\n====== { step } Step of { len(train_loader) } ======")
            accuracy = correct / total_cnt
            print(f"Train Acc : {accuracy }")
            acc_record.append(accuracy)
            print(f"Train Loss : { loss.item() / batch[1].size(0) }")

        print(f"Batch {step+1} of {len(train_loader)}")

        if step == len(train_loader) -1 and len(batch[0] < 32):
          pass

    correct = 0
    total_cnt = 0

====== 1 epoch of 10 ======
Train Acc : 0.875
Train Loss : 0.012920183129608631
Batch 1 of 256
Batch 2 of 256
Batch 3 of 256
Batch 4 of 256
Batch 5 of 256
Batch 6 of 256
Batch 7 of 256
Batch 8 of 256
Batch 9 of 256
Batch 10 of 256
Batch 11 of 256
Batch 12 of 256
Batch 13 of 256
Batch 14 of 256
Batch 15 of 256
Batch 16 of 256
Batch 17 of 256
Batch 18 of 256
Batch 19 of 256
Batch 20 of 256
Batch 21 of 256
Batch 22 of 256
Batch 23 of 256
Batch 24 of 256
Batch 25 of 256
Batch 26 of 256
Batch 27 of 256
Batch 28 of 256
Batch 29 of 256
Batch 30 of 256
Batch 31 of 256
Batch 32 of 256
Batch 33 of 256
Batch 34 of 256
Batch 35 of 256
Batch 36 of 256
Batch 37 of 256
Batch 38 of 256
Batch 39 of 256
Batch 40 of 256
Batch 41 of 256
Batch 42 of 256
Batch 43 of 256
Batch 44 of 256
Batch 45 of 256
Batch 46 of 256
Batch 47 of 256
Batch 48 of 256
Batch 49 of 256
Batch 50 of 256
Batch 51 of 256
Batch 52 of 256
Batch 53 of 256
Batch 54 of 256
Batch 55 of 256
Batch 56 of 256
Batch 57 of 256
Batch 58 of 256
B

In [ ]:
print(acc_record)

[0.875, 0.875, 0.8125, 0.8125, 0.78125, 0.78125, 0.78125, 0.78125, 0.78125, 0.78125, 0.78125, 0.75, 0.75, 0.75, 0.75, 0.75, 0.75, 0.75, 0.75, 0.75, 0.71875, 0.71875, 0.71875, 0.71875, 0.71875, 0.71875, 0.71875, 0.6875, 0.6875, 0.6875, 0.6875, 0.6875, 0.6875, 0.6875, 0.6875, 0.6875, 0.6875, 0.65625, 0.65625, 0.65625, 0.65625, 0.65625, 0.65625, 0.65625, 0.65625, 0.625, 0.625, 0.625, 0.625, 0.625, 0.625, 0.625, 0.625, 0.59375, 0.5625, 0.5625, 0.5625, 0.5625, 0.5625, 0.5625, 0.53125, 0.53125, 0.53125, 0.53125, 0.53125, 0.53125, 0.53125, 0.5, 0.5, 0.40625, 0.34375, 0.78125, 0.6875, 0.9375, 0.6875, 0.625, 0.8125, 0.875, 0.8125, 0.78125, 0.71875]


In [ ]:
acc_record.sort(reverse = True)
print(acc_record)

[0.9375, 0.875, 0.875, 0.875, 0.8125, 0.8125, 0.8125, 0.8125, 0.78125, 0.78125, 0.78125, 0.78125, 0.78125, 0.78125, 0.78125, 0.78125, 0.78125, 0.75, 0.75, 0.75, 0.75, 0.75, 0.75, 0.75, 0.75, 0.75, 0.71875, 0.71875, 0.71875, 0.71875, 0.71875, 0.71875, 0.71875, 0.71875, 0.6875, 0.6875, 0.6875, 0.6875, 0.6875, 0.6875, 0.6875, 0.6875, 0.6875, 0.6875, 0.6875, 0.6875, 0.65625, 0.65625, 0.65625, 0.65625, 0.65625, 0.65625, 0.65625, 0.65625, 0.625, 0.625, 0.625, 0.625, 0.625, 0.625, 0.625, 0.625, 0.625, 0.59375, 0.5625, 0.5625, 0.5625, 0.5625, 0.5625, 0.5625, 0.53125, 0.53125, 0.53125, 0.53125, 0.53125, 0.53125, 0.53125, 0.5, 0.5, 0.40625, 0.34375]


In [ ]:
print(learning_rate)

0.005


In [ ]:
#epoch 50일때
model.eval()  # 모델을 평가 모드로 설정

valid_correct_top1 = 0
valid_correct_top5 = 0
valid_total = 0

with torch.no_grad():
    for valid_step, valid_batch in enumerate(val_loader):
        valid_batch[0], valid_batch[1] = valid_batch[0].to(device), valid_batch[1].to(device)

        valid_logits = model(valid_batch[0])
        valid_loss = loss_fn(valid_logits, valid_batch[1])

        _, valid_predictions = valid_logits.topk(2, dim=1)  # 상위 5개 예측을 얻음
        valid_total += valid_batch[1].size(0)

        # top-1 정확도 계산
        valid_correct_top1 += valid_predictions[:, 0].eq(valid_batch[1]).sum().item()

        # top-5 정확도 계산
        valid_correct_top5 += valid_predictions.eq(valid_batch[1].view(-1, 1)).sum().item()

valid_accuracy_top1 = valid_correct_top1 / valid_total
valid_accuracy_top5 = valid_correct_top5 / valid_total

print(f"Validation Top-1 Acc: {valid_accuracy_top1}")
print(f"Validation Top-5 Acc: {valid_accuracy_top5}")


Validation Top-1 Acc: 0.7632962464849004
Validation Top-5 Acc: 0.9644210783714391


In [ ]:
len(acc_record)

81

In [ ]:
acc_record.sort(reverse = True)
print(acc_record)
print(len(acc_record))

[0.96875, 0.9375, 0.90625, 0.90625, 0.875, 0.875, 0.875, 0.84375, 0.84375, 0.84375, 0.8125, 0.8125, 0.8125, 0.8125, 0.8125, 0.8125, 0.78125, 0.78125, 0.78125, 0.78125, 0.78125, 0.78125, 0.78125, 0.78125, 0.78125, 0.78125, 0.75, 0.75, 0.75, 0.75, 0.75, 0.75, 0.75, 0.75, 0.75, 0.75, 0.71875, 0.71875, 0.71875, 0.71875, 0.71875, 0.71875, 0.71875, 0.71875, 0.6875, 0.6875, 0.6875, 0.6875, 0.6875, 0.6875, 0.6875, 0.6875, 0.6875, 0.6875, 0.6875, 0.6875, 0.65625, 0.65625, 0.65625, 0.65625, 0.65625, 0.65625, 0.65625, 0.65625, 0.625, 0.625, 0.625, 0.625, 0.625, 0.625, 0.625, 0.625, 0.625, 0.59375, 0.5625, 0.5625, 0.5625, 0.5625, 0.5625, 0.5625, 0.53125, 0.53125, 0.53125, 0.53125, 0.53125, 0.53125, 0.53125, 0.5, 0.5, 0.40625, 0.34375]
91


In [ ]:
#epoch 80일때
model.eval()  # 모델을 평가 모드로 설정

valid_correct_top1 = 0
valid_correct_top5 = 0
valid_total = 0

with torch.no_grad():
    for valid_step, valid_batch in enumerate(val_loader):
        valid_batch[0], valid_batch[1] = valid_batch[0].to(device), valid_batch[1].to(device)

        valid_logits = model(valid_batch[0])
        valid_loss = loss_fn(valid_logits, valid_batch[1])

        _, valid_predictions = valid_logits.topk(2, dim=1)  # 상위 2개 예측을 얻음
        valid_total += valid_batch[1].size(0)

        # top-1 정확도 계산
        valid_correct_top1 += valid_predictions[:, 0].eq(valid_batch[1]).sum().item()

        # top-2 정확도 계산
        valid_correct_top5 += valid_predictions.eq(valid_batch[1].view(-1, 1)).sum().item()

valid_accuracy_top1 = valid_correct_top1 / valid_total
valid_accuracy_top5 = valid_correct_top5 / valid_total

print(f"Validation Top-1 Acc: {valid_accuracy_top1}")
print(f"Validation Top-5 Acc: {valid_accuracy_top5}")


Validation Top-1 Acc: 0.8250397359090353
Validation Top-5 Acc: 0.9812935566695195


In [ ]:
acc_record.sort(reverse = True)
print(acc_record)
print(len(acc_record))

[1.0, 0.96875, 0.96875, 0.9375, 0.9375, 0.9375, 0.9375, 0.90625, 0.90625, 0.90625, 0.90625, 0.875, 0.875, 0.875, 0.875, 0.875, 0.84375, 0.84375, 0.84375, 0.84375, 0.8125, 0.8125, 0.8125, 0.8125, 0.8125, 0.8125, 0.78125, 0.78125, 0.78125, 0.78125, 0.78125, 0.78125, 0.78125, 0.78125, 0.78125, 0.78125, 0.75, 0.75, 0.75, 0.75, 0.75, 0.75, 0.75, 0.75, 0.75, 0.75, 0.71875, 0.71875, 0.71875, 0.71875, 0.71875, 0.71875, 0.71875, 0.71875, 0.6875, 0.6875, 0.6875, 0.6875, 0.6875, 0.6875, 0.6875, 0.6875, 0.6875, 0.6875, 0.6875, 0.6875, 0.65625, 0.65625, 0.65625, 0.65625, 0.65625, 0.65625, 0.65625, 0.65625, 0.625, 0.625, 0.625, 0.625, 0.625, 0.625, 0.625, 0.625, 0.625, 0.59375, 0.5625, 0.5625, 0.5625, 0.5625, 0.5625, 0.5625, 0.53125, 0.53125, 0.53125, 0.53125, 0.53125, 0.53125, 0.53125, 0.5, 0.5, 0.40625, 0.34375]
101


In [ ]:
#epoch 100일때
model.eval()  # 모델을 평가 모드로 설정

valid_correct_top1 = 0
valid_correct_top5 = 0
valid_total = 0

with torch.no_grad():
    for valid_step, valid_batch in enumerate(val_loader):
        valid_batch[0], valid_batch[1] = valid_batch[0].to(device), valid_batch[1].to(device)

        valid_logits = model(valid_batch[0])
        valid_loss = loss_fn(valid_logits, valid_batch[1])

        _, valid_predictions = valid_logits.topk(2, dim=1)  # 상위 2개 예측을 얻음
        valid_total += valid_batch[1].size(0)

        # top-1 정확도 계산
        valid_correct_top1 += valid_predictions[:, 0].eq(valid_batch[1]).sum().item()

        # top-2 정확도 계산
        valid_correct_top5 += valid_predictions.eq(valid_batch[1].view(-1, 1)).sum().item()

valid_accuracy_top1 = valid_correct_top1 / valid_total
valid_accuracy_top5 = valid_correct_top5 / valid_total

print(f"Validation Top-1 Acc: {valid_accuracy_top1}")
print(f"Validation Top-2 Acc: {valid_accuracy_top5}")


Validation Top-1 Acc: 0.9690671231201858
Validation Top-2 Acc: 0.9975547132901332


In [ ]:
#epoch 100일때
model.eval()  # 모델을 평가 모드로 설정

valid_correct_top1 = 0
valid_correct_top5 = 0
valid_total = 0

with torch.no_grad():
    for valid_step, valid_batch in enumerate(val_loader):
        valid_batch[0], valid_batch[1] = valid_batch[0].to(device), valid_batch[1].to(device)

        valid_logits = model(valid_batch[0])
        valid_loss = loss_fn(valid_logits, valid_batch[1])

        _, valid_predictions = valid_logits.topk(2, dim=1)  # 상위 2개 예측을 얻음
        valid_total += valid_batch[1].size(0)

        # top-1 정확도 계산
        valid_correct_top1 += valid_predictions[:, 0].eq(valid_batch[1]).sum().item()

        # top-2 정확도 계산
        valid_correct_top5 += valid_predictions.eq(valid_batch[1].view(-1, 1)).sum().item()

valid_accuracy_top1 = valid_correct_top1 / valid_total
valid_accuracy_top5 = valid_correct_top5 / valid_total

print(f"Validation Top-1 Acc: {valid_accuracy_top1}")
print(f"Validation Top-2 Acc: {valid_accuracy_top5}")


Validation Top-1 Acc: 0.9690671231201858
Validation Top-2 Acc: 0.9975547132901332
